# Fitting

There are two built in fitting engines, `lmfit` and `bumps`.

Import all the packages.

In [36]:
# Import all the packages
from easyCore import np
from easyDiffractionLib.sample import Sample
from easyDiffractionLib import Phases
from easyDiffractionLib.interface import InterfaceFactory as Calculator
from easyDiffractionLib.Profiles.P1D import Instrument1DCWParameters
from easyDiffractionLib.Elements.Experiments.Pattern import Pattern1D

from easyCore.Fitting.Fitting import Fitter

import matplotlib.pyplot as plt


## Preparing the sample and data

We load up a cif file and then set the experimental parameters.

In [39]:
calculator = Calculator()
c = Phases.from_cif_file('PbSO4.cif')
S = Sample(phases=c, parameters=Instrument1DCWParameters.default(), calculator=calculator)

AttributeError: 

Load up experimental data

In [38]:
data_x, data_y, data_e = np.loadtxt('PbSO4_xrays_short.xye', unpack=True)
data_y = data_y/100.0

OSError: PbSO4_xrays_short.xye not found.

Generate the simulation y-data

In [ ]:
sim_y_data = interface.fit_func(data_x)

In [ ]:
%matplotlib notebook
plt.plot(data_x, data_y, label='Experiment (X-rays)')
plt.plot(data_x, sim_y_data, label=f'Calculations ({interface.current_interface_name})')
plt.legend()

We do not have the correct experimental parameters..

In [ ]:
S.parameters.wavelength = 1.54
S.parameters.u_resolution = 0.031
S.parameters.v_resolution = -0.052
S.parameters.w_resolution = 0.032
S.parameters.x_resolution = 0.015
S.parameters.y_resolution = 0.0

#sim_y_data = interface.fit_func(data_x)

%matplotlib notebook
plt.plot(data_x, data_y, label='Experiment (X-rays)')
plt.plot(data_x, sim_y_data, label=f'Calculations ({interface.current_interface_name})')
plt.legend()

Now we need to set the background

In [ ]:
from easyDiffractionLib.Elements.Backgrounds.Point import PointBackground, BackgroundPoint

bg = PointBackground(linked_experiment='PbSO4')

#bg.append(BackgroundPoint.from_pars(data_x[0], 166))
#bg.append(BackgroundPoint.from_pars(16, 36))
#bg.append(BackgroundPoint.from_pars(50, 120))
#bg.append(BackgroundPoint.from_pars(100, 100))
#bg.append(BackgroundPoint.from_pars(data_x[-1], 164))

bg.append(BackgroundPoint.from_pars(data_x[0], 1.66))
bg.append(BackgroundPoint.from_pars(16, 0.36))
bg.append(BackgroundPoint.from_pars(50, 1.20))
bg.append(BackgroundPoint.from_pars(100, 1.00))
bg.append(BackgroundPoint.from_pars(data_x[-1], 1.64))

S.set_background(bg)


In [ ]:
sim_y_data = interface.fit_func(data_x)

%matplotlib notebook
plt.plot(data_x, data_y, label='Experiment (X-rays)')
plt.plot(data_x, sim_y_data, label=f'Calculations ({interface.current_interface_name})')
plt.legend()

Select CrysFML calculator

In [ ]:
print(interface.available_interfaces)
print(interface.current_interface_name)
interface.switch('CrysFML')
print(interface.current_interface_name)

In [ ]:
sim_y_data = interface.fit_func(data_x)

%matplotlib notebook
plt.plot(data_x, data_y, label='Experiment (X-rays)')
plt.plot(data_x, sim_y_data, label=f'Calculations ({interface.current_interface_name})')
plt.legend()

Change scale manually

In [ ]:
S.pattern.scale = 0.1
sim_y_data = interface.fit_func(data_x)

%matplotlib notebook
plt.plot(data_x, data_y, label='Experiment (X-rays)')
plt.plot(data_x, sim_y_data, label=f'Calculations ({interface.current_interface_name})')
plt.legend()

## Fitting to the data

Initalize the fitting engine and apply a few constraints

In [ ]:
f = Fitter(S, interface.fit_func)

# Vary the scale and the BG points
S.pattern.scale.fixed = False
S.pattern.zero_shift.fixed = False
#S.parameters.resolution_u.fixed = False
#S.parameters.resolution_v.fixed = False
#S.parameters.resolution_w.fixed = False
#S.parameters.resolution_x.fixed = False
#S.backgrounds[0][0].y.fixed = False
#S.backgrounds[0][1].y.fixed = False
#S.backgrounds[0][2].y.fixed = False
#S.backgrounds[0][3].y.fixed = False
#S.backgrounds[0][4].y.fixed = False

Select bumps minimizer

In [ ]:
print("available minimizers:", f.available_engines)
print()
print("current minimizer:", f.current_engine.name)
print("available methods of current minimizer:", f.available_methods())
print()
print("switch minimizer")
f.switch_engine('bumps')
f_method = 'lm'
print("current minimizer:", f.current_engine.name)
print("available methods of current minimizer:", f.available_methods())

Perform the fit

In [ ]:
result = f.fit(data_x, data_y, weights=1/data_e, method=f_method)

In [ ]:
if result.success:
    print("The fit has been successful: {}".format(result.success))
    print("The gooodness of fit is: {}".format(result.goodness_of_fit))
    
sim_y_data = interface.fit_func(data_x)

In [ ]:
%matplotlib notebook
plt.plot(data_x, data_y, label='Experimental')
plt.plot(data_x, sim_y_data, label='Best Fit')
plt.legend()

In [ ]:
print(f'Scale: {S.pattern.scale}')
print(f'Scale: {S.pattern.zero_shift}')